# Task 1: Data Exploration and Enrichment

## Objective
Understand the starter dataset and enrich it with additional data useful for forecasting financial inclusion in Ethiopia.

## Environment Setup
We use a virtual environment with `pandas`, `openpyxl`, and other required libraries.

In [ ]:
import pandas as pd
import numpy as np
import os

# Set file paths (using CSVs created during enrichment process)
data_path = "../data/raw/ethiopia_fi_unified_data.csv"
ref_codes_path = "../data/raw/reference_codes.csv"
impact_links_path = "../data/raw/impact_links.csv"

# Load data
try:
    data_df = pd.read_csv(data_path)
    impact_links_df = pd.read_csv(impact_links_path)
    ref_codes_df = pd.read_csv(ref_codes_path)
    
    print("Data loaded successfully.")
    print(f"Data Records: {len(data_df)}")
    print(f"Impact Links: {len(impact_links_df)}")
except Exception as e:
    print(f"Error loading data: {e}")

## 1. Schema Exploration

The dataset follows a unified schema where `record_type` defines how to interpret each row.

In [ ]:
print("Records by Type:")
print(data_df['record_type'].value_counts())

print("\nRecords by Pillar:")
print(data_df['pillar'].value_counts(dropna=False))

## 2. Enrichment Summary
We have enriched the dataset with the following:
- **2011 Baseline**: Account ownership started at 14%.
- **Gender Disaggregation**: Explicitly added male/female ownership rates for 2021 and 2024.
- **Operator Growth**: Added Telebirr (54M) and M-Pesa (10M) user counts for 2024.

In [ ]:
# Filter for key indicators
indicators = ['ACC_OWNERSHIP', 'ACC_MM_ACCOUNT', 'GEN_GAP_ACC']
key_obs = data_df[(data_df['record_type'] == 'observation') & (data_df['indicator_code'].isin(indicators))]
key_obs = key_obs.sort_values(['indicator_code', 'observation_date'])

display(key_obs[['indicator_code', 'observation_date', 'value_numeric', 'pillar', 'source_name']])

## 3. Event Analysis
Events are categorized by type. Their impact is modeled through `impact_link` records.

In [ ]:
print("Cataloged Events:")
events = data_df[data_df['record_type'] == 'event']
display(events[['indicator', 'category', 'observation_date']])

## Challenges in Event Pillar Assignment
Assigning a single 'pillar' to an event (e.g. Telebirr Launch) is problematic because an event often impacts multiple pillars (Access, Usage, Affordability) simultaneously. The unified schema handles this by using `impact_link` records to map one event to multiple indicators.